<a href="https://colab.research.google.com/github/ranyaa1/Flower-recognition-using-deep-learning/blob/main/softavera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install psycopg2

In [ ]:
!pip install sqlalchemy

In [ ]:
import pandas as pd
import numpy  as np
import psycopg2
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from kneed import KneeLocator
import skfuzzy as fuzz
from sklearn.metrics import silhouette_score # Importing the silhouette_score function from sklearn.metrics
from sklearn.metrics import davies_bouldin_score


In [ ]:
import psycopg2
import pandas as pd

# Connexion à la base PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",         # ou le vrai nom de ta base
    user="postgres",
    password="1234",
    host="localhost",          # ou l’IP de ton serveur
    port="5432"
)

# Créer un curseur
cur = conn.cursor()

# Requête SQL
query = "SELECT * FROM frch_stage.client_boutique;"

# Lire les résultats dans un DataFrame pandas
df = pd.read_sql_query(query, conn)

# Afficher les premières lignes
print(df.head())

# Fermer la connexion
cur.close()
conn.close()


In [ ]:
df

# **Exploratory data analysis**

**Missing values**

**RFM Table Preparation**

In [ ]:

# ----------- Calcul du RFM -----------
# Convertir 'last_order' en datetime
df['last_order'] = pd.to_datetime(df['last_order'], errors='coerce')

# Obtenir la date actuelle et la date limite des 10 dernières années
now = pd.to_datetime('today').date()
ten_years_ago = now.replace(year=now.year - 10)  # Date limite il y a 10 ans

# Filtrer les données des 10 dernières années
df_filtered = df[df['last_order'].dt.date >= ten_years_ago]

# Calculer la récence (R)
recency = df_filtered.groupby('id_client')['last_order'].max().reset_index()
recency['LastPurchaseDate'] = recency['last_order'].dt.date
recency['Recency'] = (now - recency['LastPurchaseDate']).apply(lambda x: x.days)

# Calculer la fréquence (F) et le montant total (M)
frequency = df_filtered.groupby('id_client')['total_order'].sum().reset_index()
monetary = df_filtered.groupby('id_client')['revenue'].sum().reset_index()

# Fusionner les résultats pour obtenir le tableau RFM
rfm_df = recency[['id_client', 'Recency']].merge(frequency, on='id_client').merge(monetary, on='id_client')

# Renommer les colonnes
rfm_df.rename(columns={'Recency': 'Recency',
                       'total_order': 'Frequency',
                       'revenue': 'Monetary'},
              inplace=True)

# Afficher les 5 premières lignes
print(rfm_df.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Cellule 1: Boxplot et Distplot pour Recency
plt.figure(figsize=(18, 6))

# Boxplot pour Recency
plt.subplot(1, 2, 1)
sns.boxplot(y=rfm_df['Recency'])
plt.title('Boxplot de la Récence')

# Distplot pour Recency
plt.subplot(1, 2, 2)
sns.histplot(rfm_df['Recency'], kde=True)
plt.title('Distribution de la Récence')

plt.tight_layout()
plt.show()


In [ ]:
# Cellule 2: Boxplot et Distplot pour Frequency
plt.figure(figsize=(18, 6))

# Boxplot pour Frequency
plt.subplot(1, 2, 1)
sns.boxplot(y=rfm_df['Frequency'])
plt.title('Boxplot de la Fréquence')

# Distplot pour Frequency
plt.subplot(1, 2, 2)
sns.histplot(rfm_df['Frequency'], kde=True)
plt.title('Distribution de la Fréquence')

plt.tight_layout()
plt.show()


In [ ]:
# Cellule 3: Boxplot et Distplot pour Monetary
plt.figure(figsize=(18, 6))

# Boxplot pour Monetary
plt.subplot(1, 2, 1)
sns.boxplot(y=rfm_df['Monetary'])
plt.title('Boxplot du Montant')

# Distplot pour Monetary
plt.subplot(1, 2, 2)
sns.histplot(rfm_df['Monetary'], kde=True)
plt.title('Distribution du Montant')

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Étape 1 : Calcul des bornes IQR
Q1 = rfm_df['Frequency'].quantile(0.25)
Q3 = rfm_df['Frequency'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Étape 2 : Remplacement des outliers par la moyenne
mean_value = rfm_df['Frequency'].mean()
rfm_df['Frequency_corrected'] = rfm_df['Frequency'].apply(
    lambda x: mean_value if x < lower_bound or x > upper_bound else x
)

# Étape 3 : Affichage des boxplot et distplot
plt.figure(figsize=(18, 6))

# Boxplot
plt.subplot(1, 2, 1)
sns.boxplot(y=rfm_df['Frequency_corrected'])
plt.title('Boxplot corrigé de la Fréquence (mean)')

# Distplot
plt.subplot(1, 2, 2)
sns.histplot(rfm_df['Frequency_corrected'], kde=True)
plt.title('Distribution corrigée de la Fréquence (mean)')

plt.tight_layout()
plt.show()


In [ ]:
# Étape 1 : Calcul des bornes IQR pour Monetary
Q1_m = rfm_df['Monetary'].quantile(0.25)
Q3_m = rfm_df['Monetary'].quantile(0.75)
IQR_m = Q3_m - Q1_m
lower_bound_m = Q1_m - 1.5 * IQR_m
upper_bound_m = Q3_m + 1.5 * IQR_m

# Étape 2 : Appliquer le capping/flooring
rfm_df['Monetary_capped'] = rfm_df['Monetary'].apply(
    lambda x: lower_bound_m if x < lower_bound_m else upper_bound_m if x > upper_bound_m else x
)

# Étape 3 : Affichage boxplot + distplot
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(18, 6))

# Boxplot
plt.subplot(1, 2, 1)
sns.boxplot(y=rfm_df['Monetary_capped'])
plt.title('Boxplot - Monetary (Capping/Flooring)')

# Distplot
plt.subplot(1, 2, 2)
sns.histplot(rfm_df['Monetary_capped'], kde=True)
plt.title('Distribution - Monetary (Capping/Flooring)')

plt.tight_layout()
plt.show()


In [ ]:

# heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(rfm_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")

plt.title('Heatmap of Correlation Matrix for RFM Data')
plt.show()

In [ ]:

# plot the distribution of RFM values
f,ax = plt.subplots(figsize=(15, 15))
plt.subplot(3, 1, 1); sns.distplot(rfm_df.Recency, label = 'Recency')
plt.subplot(3, 1, 2); sns.distplot(rfm_df.Frequency, label = 'Frequency')
plt.subplot(3, 1, 3); sns.distplot(rfm_df.Monetary, label = 'Monetary')
plt.style.use('fivethirtyeight')
plt.tight_layout()
plt.show()

# **Features engineering**

**Feature Transformation**

In [ ]:
rfm_df

In [ ]:
pip install kneed

In [ ]:

# Ignorer les warnings
warnings.filterwarnings("ignore")

# Standardisation
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_df)

feature extraction


In [ ]:


# PCA à 2 dimensions
pca = PCA(n_components=2)
rfm_pca = pca.fit_transform(rfm_scaled)
rfm_pca_df = pd.DataFrame(rfm_pca, columns=['PCA1', 'PCA2'])


kmean

In [ ]:

# Calcule de WCSS
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(rfm_pca_df)
    wcss.append(kmeans.inertia_)

# Trouver l'optimal cluster avec KneeLocator
kl = KneeLocator(range(1, 11), wcss, curve='convex', direction='decreasing')
elbow_point = kl.elbow

# Tracer la courbe du coude
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='-', color='b', label='WCSS')
plt.axvline(x=elbow_point, color='red', linestyle='--', label=f'Elbow = {elbow_point}')
plt.scatter(elbow_point, wcss[elbow_point-1], color='red', s=100)
plt.title('Elbow Method with Optimal Cluster')
plt.xlabel('Number of Clusters')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.xticks(range(1, 11))
plt.grid(True)
plt.legend()
plt.show()


In [ ]:

# Silhouette Score pour chaque k
silhouette_scores = []
k_range = range(2, 11)

print("Silhouette Scores by number of clusters:")
for k in k_range:
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(rfm_pca_df)
    labels = kmeans.labels_
    score = silhouette_score(rfm_pca_df, labels)
    silhouette_scores.append(score)
    print(f"k = {k}: Silhouette Score = {score:.4f}")

# Tracer le graphique
plt.figure(figsize=(10, 6))
plt.plot(k_range, silhouette_scores, marker='o', linestyle='-', color='green')
plt.title('Silhouette Score vs Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.xticks(k_range)
plt.grid(True)
plt.show()


In [ ]:

# Listes pour stocker les scores
silhouette_scores = []
db_scores = []

# Plage des valeurs de k à tester
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_pca_df)
    labels = kmeans.labels_

    silhouette = silhouette_score(rfm_pca_df, labels)
    db = davies_bouldin_score(rfm_pca_df, labels)

    silhouette_scores.append(silhouette)
    db_scores.append(db)

# Tracer les deux scores sur le même graphique
sns.set(style="whitegrid")
fig, ax1 = plt.subplots(figsize=(10, 6))

# Axe pour Silhouette Score
color = 'tab:green'
ax1.set_xlabel('Nombre de clusters (k)')
ax1.set_ylabel('Silhouette Score', color=color)
ax1.plot(K_range, silhouette_scores, marker='o', color=color, label='Silhouette Score')
ax1.tick_params(axis='y', labelcolor=color)

# Deuxième axe y pour Davies-Bouldin Score
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Davies-Bouldin Score', color=color)
ax2.plot(K_range, db_scores, marker='s', color=color, label='Davies-Bouldin Score')
ax2.tick_params(axis='y', labelcolor=color)

# Titre et affichage
plt.title('Comparaison des scores Silhouette et Davies-Bouldin selon k')
plt.xticks(K_range)
plt.show()


In [ ]:
# Afficher les scores
print("Scores pour chaque valeur de k:")
print(f"{'k':<5}{'Silhouette Score':<25}{'Davies-Bouldin Score'}")
for k, sil, db in zip(K_range, silhouette_scores, db_scores):
    print(f"{k:<5}{sil:<25.4f}{db:.4f}")

In [ ]:
pip install pyclustering


In [ ]:
pip install sklearn-extra pyclustering scikit-fuzzy


Fuzzy C-Means

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz

# List to store inertia values
inertia_scores_fcm = []
k_range = range(2, 11)

# Calculate Fuzzy C-Means and inertia
for k in k_range:
    # Apply Fuzzy C-Means
    cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(rfm_pca_df.values.T, k, 2, error=1e-5, maxiter=1000)

    # Compute inertia for Fuzzy C-Means (sum of squared distances to the cluster centers)
    fcm_labels = np.argmax(u, axis=0)
    inertia = np.sum((rfm_pca_df.values - cntr[fcm_labels])**2)

    # Store inertia value
    inertia_scores_fcm.append(inertia)

# Plot the Elbow Method for Fuzzy C-Means (Inertia)
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(k_range, inertia_scores_fcm, marker='o', linestyle='-', color='green', label='Inertia (Fuzzy C-Means)')
ax.set_xlabel('Number of Clusters (k)')
ax.set_ylabel('Inertia')
ax.set_title('Elbow Method for Fuzzy C-Means')
plt.grid(True)
plt.show()


In [ ]:

silhouette_scores_fcm = []
db_scores_fcm = []
k_range = range(2, 11)

print(f"{'k':<5}{'Silhouette Score':<25}{'Davies-Bouldin Score'}")
for k in k_range:
    # Appliquer Fuzzy C-Means
    cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(rfm_pca_df.values.T, k, 2, error=1e-5, maxiter=1000)

    fcm_labels = np.argmax(u, axis=0)

    sil_score = silhouette_score(rfm_pca_df, fcm_labels)
    db_score = davies_bouldin_score(rfm_pca_df, fcm_labels)

    silhouette_scores_fcm.append(sil_score)
    db_scores_fcm.append(db_score)

    print(f"{k:<5}{sil_score:<25.4f}{db_score:.4f}")

# Tracer les courbes
fig, ax1 = plt.subplots(figsize=(10, 6))

# Axe pour Silhouette Score
color = 'blue'
ax1.set_xlabel('Nombre de clusters (k)')
ax1.set_ylabel('Silhouette Score', color=color)
ax1.plot(k_range, silhouette_scores_fcm, marker='o', linestyle='-', color=color, label='Silhouette Score')
ax1.tick_params(axis='y', labelcolor=color)

# Deuxième axe y pour Davies-Bouldin Score
ax2 = ax1.twinx()
color = 'red'
ax2.set_ylabel('Davies-Bouldin Score', color=color)
ax2.plot(k_range, db_scores_fcm, marker='s', linestyle='--', color=color, label='Davies-Bouldin Score')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Fuzzy C-Means: Silhouette vs Davies-Bouldin Scores')
plt.xticks(k_range)
plt.grid(True)
plt.show()


Mean Shift

In [ ]:
from sklearn.cluster import MeanShift # Import MeanShift

meanshift = MeanShift()
labels_meanshift = meanshift.fit_predict(rfm_pca_df)

# Calcul des scores
silhouette_avg = silhouette_score(rfm_pca_df, labels_meanshift)
db_score = davies_bouldin_score(rfm_pca_df, labels_meanshift)

# Affichage des scores
print(f"Mean Shift Clustering Results:")
print(f"- Silhouette Score         : {silhouette_avg:.4f}")
print(f"- Davies-Bouldin Score     : {db_score:.4f}")
print(f"- Nombre de clusters trouvés : {len(set(labels_meanshift))}")

# Visualisation
plt.figure(figsize=(8, 6))
plt.scatter(rfm_pca_df['PCA1'], rfm_pca_df['PCA2'], c=labels_meanshift, cmap='viridis', s=50)
plt.title("Mean Shift Clustering")
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.grid(True)
plt.show()


In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt

sil_scores = []
db_scores = []
n_range = range(2, 11)

print("Scores pour GMM (Gaussian Mixture Model):")
for n in n_range:
    gmm = GaussianMixture(n_components=n, random_state=42)
    labels = gmm.fit_predict(rfm_pca_df)

    sil = silhouette_score(rfm_pca_df, labels)
    db = davies_bouldin_score(rfm_pca_df, labels)

    sil_scores.append(sil)
    db_scores.append(db)

    print(f"n_components = {n}: Silhouette = {sil:.4f}, Davies-Bouldin = {db:.4f}")

# Visualisation
fig, ax1 = plt.subplots(figsize=(10, 6))

# Axe gauche : Silhouette Score
color = 'tab:blue'
ax1.set_xlabel('Number of Components')
ax1.set_ylabel('Silhouette Score', color=color)
ax1.plot(n_range, sil_scores, marker='o', color=color, label='Silhouette Score')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:orange'
ax2.set_ylabel('Davies-Bouldin Score', color=color)
ax2.plot(n_range, db_scores, marker='s', color=color, label='Davies-Bouldin Score')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Silhouette vs Davies-Bouldin Scores (GMM)')
plt.grid(True)
plt.xticks(n_range)
plt.show()


In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Tester plusieurs valeurs de k
silhouette_scores = []
k_range = range(2, 11)

print("Silhouette Scores (Spectral Clustering):")
for k in k_range:
    spectral = SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=42)
    labels = spectral.fit_predict(rfm_pca_df)
    score = silhouette_score(rfm_pca_df, labels)
    silhouette_scores.append(score)
    print(f"k = {k}: Silhouette Score = {score:.4f}")

# Tracer la courbe
plt.figure(figsize=(10, 6))
plt.plot(k_range, silhouette_scores, marker='o', linestyle='-', color='purple')
plt.title('Silhouette Score vs Number of Clusters (Spectral Clustering)')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.show()


In [ ]:
pip install scikit-learn-extra


In [ ]:
!pip install --upgrade numpy

In [ ]:
!pip install --upgrade --force-reinstall sklearn-extra

In [ ]:
!pip install --upgrade --force-reinstall scikit-learn

In [ ]:
!pip install numpy==1.26.4
!pip install --upgrade --force-reinstall scikit-learn-extra


In [ ]:
from sklearn_extra.cluster import KMedoids
from kneed import KneeLocator
import matplotlib.pyplot as plt
import numpy as np

# Liste pour stocker la "pseudo-inertia" (somme des distances intra-cluster)
inertia_kmedoids = []
k_range = range(1, 11)

for k in k_range:
    kmedoids = KMedoids(n_clusters=k, random_state=42, method='pam')
    kmedoids.fit(rfm_pca_df)

    # Calcule la somme des distances intra-cluster (médiane -> points affectés)
    total_distance = 0
    for i in range(k):
        cluster_points = rfm_pca_df[kmedoids.labels_ == i]
        medoid = kmedoids.cluster_centers_[i]
        total_distance += np.sum(np.linalg.norm(cluster_points - medoid, axis=1))

    inertia_kmedoids.append(total_distance)
,
# Trouver le coude
kl = KneeLocator(k_range, inertia_kmedoids, curve='convex', direction='decreasing')
elbow_k = kl.elbow

# Tracer la courbe du coude
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia_kmedoids, marker='o', linestyle='-', color='purple', label='Total Intra-Cluster Distance')
if elbow_k is not None:
    plt.axvline(x=elbow_k, color='red', linestyle='--', label=f'Elbow = {elbow_k}')
    plt.scatter(elbow_k, inertia_kmedoids[elbow_k - 1], color='red', s=100)
plt.title('Elbow Method for K-Medoids (Pseudo-Inertia)')
plt.xlabel('Number of Clusters')
plt.ylabel('Total Intra-Cluster Distance')
plt.xticks(k_range)
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Appliquer KMeans pour k=2
kmeans_2 = KMeans(n_clusters=2, init='random', random_state=42)
clusters_2 = kmeans_2.fit_predict(rfm_pca_df)
silhouette_2 = silhouette_score(rfm_pca_df, clusters_2)

# Appliquer KMeans pour k=3
kmeans_3 = KMeans(n_clusters=3, init='random', random_state=42)
clusters_3 = kmeans_3.fit_predict(rfm_pca_df)
silhouette_3 = silhouette_score(rfm_pca_df, clusters_3)

# Créer deux subplots côte à côte
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Visualisation pour k=2
ax1.scatter(rfm_pca_df['PCA1'], rfm_pca_df['PCA2'], c=clusters_2, cmap='Set2', s=50)
ax1.set_title(f'Clusters (k=2)\nSilhouette = {silhouette_2:.3f}')
ax1.set_xlabel('PCA1')
ax1.set_ylabel('PCA2')
ax1.grid(True)

# Visualisation pour k=3
ax2.scatter(rfm_pca_df['PCA1'], rfm_pca_df['PCA2'], c=clusters_3, cmap='Set1', s=50)
ax2.set_title(f'Clusters (k=3)\nSilhouette = {silhouette_3:.3f}')
ax2.set_xlabel('PCA1')
ax2.set_ylabel('PCA2')
ax2.grid(True)

plt.suptitle('Comparaison des clusters avec PCA (k=2 vs k=3)', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Normalisation des données
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_df)

# Architecture de l'autoencoder améliorée
input_dim = rfm_scaled.shape[1]
input_layer = Input(shape=(input_dim,))
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)
bottleneck = Dense(2, activation='tanh')(x)
x = Dense(32, activation='relu')(bottleneck)
x = Dense(64, activation='relu')(x)
output_layer = Dense(input_dim, activation='linear')(x)

# Modèle Autoencoder
autoencoder = Model(inputs=input_layer, outputs=output_layer)
encoder = Model(inputs=input_layer, outputs=bottleneck)

autoencoder.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
autoencoder.fit(rfm_scaled, rfm_scaled, epochs=100, batch_size=32, shuffle=True, verbose=1)

# Obtenir l'espace latent
rfm_autoencoded = encoder.predict(rfm_scaled)

# Clustering avec KMeans
kmeans_2 = KMeans(n_clusters=2, init='random', random_state=42)
clusters_2 = kmeans_2.fit_predict(rfm_autoencoded)

kmeans_3 = KMeans(n_clusters=3, init='random', random_state=42)
clusters_3 = kmeans_3.fit_predict(rfm_autoencoded)

# Silhouette scores
score_k2 = silhouette_score(rfm_autoencoded, clusters_2)
score_k3 = silhouette_score(rfm_autoencoded, clusters_3)
print("Silhouette score (Autoencoder + k=2):", round(score_k2, 3))
print("Silhouette score (Autoencoder + k=3):", round(score_k3, 3))

# Visualisation avec seaborn
df_latent = pd.DataFrame(rfm_autoencoded, columns=['Latent 1', 'Latent 2'])
df_latent['Cluster_k2'] = clusters_2
df_latent['Cluster_k3'] = clusters_3

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.scatterplot(data=df_latent, x='Latent 1', y='Latent 2', hue='Cluster_k2', palette='Set2', s=60, ax=axes[0])
axes[0].set_title('Clusters avec Autoencoder (k=2)')
axes[0].grid(True)

sns.scatterplot(data=df_latent, x='Latent 1', y='Latent 2', hue='Cluster_k3', palette='Set1', s=60, ax=axes[1])
axes[1].set_title('Clusters avec Autoencoder (k=3)')
axes[1].grid(True)

plt.suptitle('Comparaison des clusters Autoencoder (k=2 vs k=3)', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
pip install umap-learn


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Liste pour stocker les scores
silhouette_scores = []

# Tester différents nombres de clusters
k_range = range(2, 11)

print("Silhouette Scores (Clustering Hiérarchique) :")
for k in k_range:
    hc = AgglomerativeClustering(n_clusters=k)
    labels = hc.fit_predict(rfm_pca_df)
    score = silhouette_score(rfm_pca_df, labels)
    silhouette_scores.append(score)
    print(f"k = {k}: Silhouette Score = {score:.4f}")

# Tracer les scores
plt.figure(figsize=(10, 6))
plt.plot(k_range, silhouette_scores, marker='o', linestyle='-', color='purple')
plt.title('Silhouette Score vs Number of Clusters (Hierarchical Clustering)')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.xticks(k_range)
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering

silhouette_scores = []

for n_clusters in range(2, 11):  # Tester les clusters entre 2 et 10
    agg_clust = AgglomerativeClustering(n_clusters=n_clusters, linkage='complete')
    labels = agg_clust.fit_predict(rfm_pca_df)
    silhouette_avg = silhouette_score(rfm_pca_df, labels)
    silhouette_scores.append(silhouette_avg)
# Afficher les valeurs de silhouette scores pour chaque k
for k, score in zip(K_range, silhouette_scores):
    print(f"Pour k = {k}, le score de silhouette est {score:.4f}")

# Afficher les scores de silhouette pour chaque k
plt.figure(figsize=(8, 5))
plt.plot(range(2, 11), silhouette_scores, marker='o', color='b')
plt.title('Scores de Silhouette pour différents nombres de clusters')
plt.xlabel('Nombre de Clusters')
plt.ylabel('Score de Silhouette')
plt.show()


In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

# Essayer plusieurs valeurs de k pour GMM
silhouette_scores = []
k_range = range(2, 11)  # Tester de 2 à 10 clusters

for k in k_range:
    gmm = GaussianMixture(n_components=k, random_state=42)
    gmm_labels = gmm.fit_predict(rfm_pca_df)
    score = silhouette_score(rfm_pca_df, gmm_labels)
    silhouette_scores.append(score)

# Trouver le k optimal avec le meilleur score de silhouette
optimal_k = k_range[silhouette_scores.index(max(silhouette_scores))]
print(f"Le nombre optimal de clusters selon le score de silhouette est : {optimal_k}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# ----------------------------
# 1. Chargement des données RFM
# ----------------------------
# Exemple : si tu as déjà ton rfm_df, sinon charge-le
# rfm_df = pd.read_csv('chemin/vers/tes_donnees.csv')

# ----------------------------
# 2. Normalisation des données
# ----------------------------
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_df)

# ----------------------------
# 3. Autoencoder
# ----------------------------
input_dim = rfm_scaled.shape[1]
input_layer = Input(shape=(input_dim,))
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)
bottleneck = Dense(2, activation='tanh')(x)
x = Dense(32, activation='relu')(bottleneck)
x = Dense(64, activation='relu')(x)
output_layer = Dense(input_dim, activation='linear')(x)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
encoder = Model(inputs=input_layer, outputs=bottleneck)

autoencoder.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
autoencoder.fit(rfm_scaled, rfm_scaled, epochs=100, batch_size=32, shuffle=True, verbose=1)

# ----------------------------
# 4. Réduction dimensionnelle
# ----------------------------
rfm_autoencoded = encoder.predict(rfm_scaled)

# ----------------------------
# 5. Clustering avec KMeans (k=3)
# ----------------------------
kmeans = KMeans(n_clusters=3, init='random', random_state=42)
clusters = kmeans.fit_predict(rfm_autoencoded)

# Ajouter les clusters à la DataFrame d'origine
rfm_df['Cluster'] = clusters

# ----------------------------
# 6. Silhouette Score
# ----------------------------
sil_score = silhouette_score(rfm_autoencoded, clusters)
print(f"Silhouette Score pour k=3 : {round(sil_score, 4)}")

# ----------------------------
# 7. Visualisation des clusters
# ----------------------------
df_latent = pd.DataFrame(rfm_autoencoded, columns=['Latent 1', 'Latent 2'])
df_latent['Cluster'] = clusters

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_latent, x='Latent 1', y='Latent 2', hue='Cluster', palette='Set1', s=70)
plt.title('Clusters Clients avec Autoencoder + KMeans (k=3)')
plt.grid(True)
plt.show()

# ----------------------------
# 8. Analyse des segments
# ----------------------------
cluster_profiles = rfm_df.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()
print("\nProfil des segments clients :")
print(cluster_profiles)


In [ ]:
# Moyennes par cluster
cluster_summary = rfm_df.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).sort_index()

print(cluster_summary)

In [ ]:
segment_labels = {
    0: 'Inactifs / perdus',
    1: 'Occasionnels',
    2: 'VIP / fidèles'
}

rfm_df['Segment_Label'] = rfm_df['Cluster'].map(segment_labels)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Count the number of clients per segment
segment_counts = rfm_df['Segment_Label'].value_counts()

# Display as a table
print(segment_counts)

# Display as a chart
plt.figure(figsize=(8, 5))
sns.countplot(data=rfm_df, x='Segment_Label', order=segment_counts.index, palette='Set2')
plt.title('Distribution of Customers by Segment')
plt.ylabel('Number of Customers')
plt.xlabel('Segment')
plt.xticks(rotation=20)
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
pip install imbalanced-learn


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Select features and labels
X = rfm_df[['Recency', 'Frequency', 'Monetary']]  # Replace this with your actual features
y = rfm_df['Cluster']  # The 'Cluster' column contains the cluster labels

# Split the data into train and test sets (optional, for validation if needed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to balance the classes in the training set
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

# Check class distribution before and after SMOTE
print("Class distribution before SMOTE:")
print(y_train.value_counts())

print("\nClass distribution after SMOTE:")
print(pd.Series(y_res).value_counts())

# Optional: Visualization to check the balance
plt.figure(figsize=(8, 5))
sns.countplot(x=y_res, palette='Set2')
plt.title('Class Distribution After SMOTE')
plt.xlabel('Cluster')
plt.ylabel('Number of Samples')
plt.xticks(rotation=20)
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Normalisation des données
scaler = StandardScaler()
X = rfm_df[['Recency', 'Frequency', 'Monetary']]  # Features
y = rfm_df['Cluster']  # Labels (clusters)

# Diviser en jeux d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE pour équilibrer les classes dans le jeu d'entraînement
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Vérification de la répartition après SMOTE
print("Répartition après SMOTE :")
print(pd.Series(y_train_res).value_counts())


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Initialiser et entraîner le modèle KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_res, y_train_res)

# Prédictions sur le jeu de test
y_pred_knn = knn.predict(X_test)

# Évaluation du modèle
print("Classification Report - KNN :")
print(classification_report(y_test, y_pred_knn))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Initialiser et entraîner le modèle Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train_res, y_train_res)

# Prédictions sur le jeu de test
y_pred_dt = dt.predict(X_test)

# Évaluation du modèle
print("Classification Report - Decision Tree :")
print(classification_report(y_test, y_pred_dt))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report

# Initialisation du modèle
model = Sequential()

# Ajouter une couche d'entrée (la première couche du réseau) et une couche cachée
model.add(Dense(units=64, activation='relu', input_dim=X_train_res.shape[1]))  # input_dim correspond au nombre de features dans tes données d'entrée

# Ajouter une couche cachée supplémentaire (si nécessaire)
model.add(Dense(units=32, activation='relu'))

# Ajouter une couche de sortie (taille de sortie dépend du nombre de classes)
model.add(Dense(units=1, activation='sigmoid'))  # Pour une classification binaire

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Utiliser 'categorical_crossentropy' pour une classification multiclasse

# Entraîner le modèle
model.fit(X_train_res, y_train_res, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Faire des prédictions
y_pred_ann = model.predict(X_test)

# Convertir les prédictions en labels binaires
y_pred_ann = (y_pred_ann > 0.5)

# Évaluer le modèle
print("Classification Report - Artificial Neural Network :")
print(classification_report(y_test, y_pred_ann))


In [ ]:
pip install xgboost


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# Initialiser et entraîner le modèle XGBoost
xgb = XGBClassifier(n_estimators=100, max_depth=10, random_state=42)
xgb.fit(X_train_res, y_train_res)

# Prédictions sur le jeu de test
y_pred_xgb = xgb.predict(X_test)

# Évaluation du modèle
print("Classification Report - XGBoost :")
print(classification_report(y_test, y_pred_xgb))


In [ ]:
print("Train Accuracy: ", train_accuracy)
print("Test Accuracy: ", test_accuracy)

In [ ]:
pip install --upgrade scikit-learn xgboost


In [ ]:
!pip uninstall scikit-learn xgboost
!pip install scikit-learn xgboost

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Définir les hyperparamètres à tester
param_grid = {
    'n_estimators': [50, 100, 150],  # Nombre d'arbres
    'max_depth': [3, 5, 10],          # Profondeur des arbres
    'learning_rate': [0.01, 0.05, 0.1],  # Taux d'apprentissage
    'subsample': [0.8, 1.0],          # Fraction d'échantillons pour chaque arbre
    'colsample_bytree': [0.8, 1.0],   # Fraction de caractéristiques pour chaque arbre
}

# Initialiser le modèle XGBoost
xgb = XGBClassifier(random_state=42)

# Initialiser GridSearchCV
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid,
                           cv=3, verbose=1, n_jobs=-1, scoring='accuracy')

# Exécuter GridSearchCV
grid_search.fit(X_train_res, y_train_res)

# Afficher les meilleurs paramètres trouvés
print("Best Parameters found: ", grid_search.best_params_)

# Obtenir le meilleur modèle
best_xgb_model = grid_search.best_estimator_

# Prédictions avec le meilleur modèle
y_pred_xgb_best = best_xgb_model.predict(X_test)

# Évaluation du modèle
from sklearn.metrics import classification_report
print("Classification Report - Best XGBoost Model:")
print(classification_report(y_test, y_pred_xgb_best))


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialiser et entraîner le modèle Random Forest
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)


rf.fit(X_train_res, y_train_res)

# Prédictions sur le jeu de test
y_pred_rf = rf.predict(X_test)

# Évaluation du modèle
print("Classification Report - Random Forest :")
print(classification_report(y_test, y_pred_rf))


In [ ]:
from sklearn.metrics import accuracy_score

# Prédictions sur l'ensemble d'entraînement
y_train_pred_rf = rf.predict(X_train_res)

# Calcul des précisions
train_accuracy = accuracy_score(y_train_res, y_train_pred_rf)
test_accuracy = accuracy_score(y_test, y_pred_rf)

print(f"Train Accuracy : {train_accuracy:.4f}")
print(f"Test Accuracy  : {test_accuracy:.4f}")

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Définir les hyperparamètres à tester
param_grid = {
    'n_estimators': [50, 100, 150],      # Nombre d'arbres
    'max_depth': [10, 20, None],          # Profondeur maximale des arbres
    'max_features': ['auto', 'sqrt'],     # Nombre de features à considérer à chaque split
    'min_samples_split': [2, 5, 10],      # Nombre minimum d'échantillons pour splitter un noeud
    'min_samples_leaf': [1, 2, 4],        # Nombre minimum d'échantillons dans un feuille
    'bootstrap': [True, False]            # Utiliser bootstrap pour la création des arbres
}

# Initialiser le modèle Random Forest
rf = RandomForestClassifier(random_state=42)

# Initialiser GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Entraîner le modèle avec GridSearchCV
grid_search.fit(X_train_res, y_train_res)

# Afficher les meilleurs paramètres trouvés
print("Best parameters found: ", grid_search.best_params_)

# Utiliser le meilleur modèle pour faire des prédictions
best_rf = grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_test)

# Évaluation du modèle
print("Classification Report - Best Random Forest :")
print(classification_report(y_test, y_pred_rf))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Initialiser le modèle Random Forest avec les meilleurs paramètres
rf = RandomForestClassifier(
    n_estimators=150,
    max_depth=10,
    bootstrap=True,
    max_features='sqrt',
    min_samples_leaf=2,
    min_samples_split=10,
    random_state=42
)

# Entraîner le modèle avec les données d'entraînement
rf.fit(X_train_res, y_train_res)

# Prédictions sur le jeu de test
y_pred_rf = rf.predict(X_test)

# Évaluation du modèle : afficher le rapport de classification
print("Classification Report - Best Random Forest :")
print(classification_report(y_test, y_pred_rf))


In [ ]:
from sklearn.metrics import accuracy_score

# Prédictions sur l'ensemble d'entraînement
y_train_pred_rf = rf.predict(X_train_res)

# Calcul des précisions
train_accuracy = accuracy_score(y_train_res, y_train_pred_rf)
test_accuracy = accuracy_score(y_test, y_pred_rf)

print(f"Train Accuracy : {train_accuracy:.4f}")
print(f"Test Accuracy  : {test_accuracy:.4f}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred_rf)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - Random Forest")
plt.grid(False)
plt.show()



LIME (Local Interpretable Model-agnostic Explanations)

In [ ]:
pip install lime


In [ ]:
from lime.lime_tabular import LimeTabularExplainer  # Import LimeTabularExplainer

# 3. Créer un explicateur LIME pour les données tabulaires
explainer = LimeTabularExplainer(
    training_data=X_train.values,
    mode="classification",
    training_labels=y_train.values,
    feature_names=X.columns,
    class_names=np.unique(y_train),
    discretize_continuous=True
)

# 4. Choisir un échantillon pour lequel nous voulons expliquer la prédiction
idx = 10  # Par exemple, on choisit la 10e observation du jeu de test
data_point = X_test.iloc[idx]

# 5. Expliquer la prédiction pour cet échantillon
explanation = explainer.explain_instance(data_point.values, rf.predict_proba, num_features=3)

# 6. Visualiser l'explication
explanation.show_in_notebook(show_table=True, show_all=False)

# 7. (Optionnel) Imprimer les résultats de la prédiction pour ce point
print(f"Prediction: {rf.predict([data_point])[0]}")
print(f"True label: {y_test.iloc[idx]}")

# 8. Évaluation du modèle
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))